## Process data & Join with Target variabel

Data streches from 22-05 -23-12, if its not 90Evers the rest of the data is excluded since we cannot know the outcome

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sqlalchemy as sa
import pyodbc
import warnings
import urllib
warnings.filterwarnings('ignore')



################### GET PAYMENT DATA ##########################

path = "../2. Python/Classes"
os.chdir(path)

from Preprocessing_Application_N import DataPreprocessor


# Define the path to the .env file
desktop_path = os.path.join(os.environ['HOME'], 'Desktop')
folder_path = os.path.join(desktop_path, 'L')
env_file_path = os.path.join(folder_path, '.env')

# Function to load credentials from a .env file
def load_credentials(file_path):
    credentials = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            credentials[key] = value
    return credentials

# Load the credentials
credentials = load_credentials(env_file_path)


In [2]:

# Initialize DataPreprocessor with all required parameters, including the driver
processor = DataPreprocessor(credentials['server'], credentials['nim_backup_db'], credentials['username'], credentials['password'], credentials['driver'])

print(os.getcwd())


path = "../../1. Data/SSN_Application - Target.sql"


df = processor.fetch_data_from_sql(path)


/Users/andreasnilsson/Library/CloudStorage/OneDrive-Nstart/Skrivbordet/Repository Homes/PSD2-Data-Engineering-Modeling-DATA-258-1/2. Python/Classes


In [3]:
df.sort_values(by='SSN')

,ReceivedDate,ApplicationID,AccountNumber,SSN,DisbursedDate,Amount,IsMainApplicant,ApplicantNo,HasCoapp,Ever90
4983,2023-07-10,8746736,7933575,0001011246,2023-07-11,177000.0,1,1,0,0
1192,2022-05-23,8264991,7757552,0001011709,2022-05-31,305000.0,1,1,1,0
5694,2023-10-13,8852723,7969710,0001016955,2023-10-18,32000.0,1,1,1,0
5077,2023-07-27,8764638,7939911,0001021096,2023-08-01,145000.0,1,1,0,0
3183,2022-12-07,8493347,7848427,0001031939,2022-12-14,310000.0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...
1580,2022-06-30,8299451,7773658,9912214393,2022-07-07,225000.0,1,1,0,0
3447,2023-01-09,8528109,7862196,9912226405,2023-01-13,180000.0,1,1,0,1
5034,2023-07-18,8756643,7937071,9912233906,2023-07-20,200000.0,1,1,0,0
1743,2022-07-18,8317664,7780794,9912270692,2022-07-19,125000.0,1,1,0,0


In [4]:
df.Ever90.value_counts()

Ever90
0    5506
1     423
Name: count, dtype: int64

In [5]:
df[df.SSN == '9001144832']

,ReceivedDate,ApplicationID,AccountNumber,SSN,DisbursedDate,Amount,IsMainApplicant,ApplicantNo,HasCoapp,Ever90
454,2022-03-17,8212497,7732407,9001144832,2022-03-18,300000.0,1,1,1,0
1259,2022-06-01,8273573,7759988,9001144832,2022-06-07,330000.0,1,1,1,0


In [6]:
df.SSN

0       6305211465
1       7909276276
2       6502020784
3       7107216629
4       7804090467
           ...    
5924    6501038605
5925    8609217537
5926    8612276900
5927    9310119012
5928    8810144868
Name: SSN, Length: 5929, dtype: object

In [7]:
path = "../../1. Data/Cleaned Transaction Data"
os.chdir(path)

In [8]:
os.getcwd()

'/Users/andreasnilsson/Library/CloudStorage/OneDrive-Nstart/Skrivbordet/Repository Homes/PSD2-Data-Engineering-Modeling-DATA-258-1/1. Data/Cleaned Transaction Data'

In [9]:
transactions = pd.read_excel('Transactions_Ink Nim Calculations 0-6132.xlsx')
len(set(transactions.SSN))

5709

In [10]:
transactions = transactions.drop_duplicates(subset='SSN', keep='first')

In [11]:
transactions['SSN'] = transactions['SSN'].astype(str).str.slice(2)

In [12]:
transactions

,NR,SSN,avg_CashWithdrawal,avg_Crypto,avg_DebtCollection,avg_Deposit,avg_DirectDebit,avg_Gambling,avg_Gaming,avg_HousingCost,...,Unclassified_partOfSalary6Months,Unclassified_partOfSalary9Months,Unclassified_sum12Months,Unclassified_sum3Months,Unclassified_sum6Months,Unclassified_sum9Months,sum creditors,sum debtCollectors,count creditors,count debtCollectors
0,60,0004046850,2657,0,176,273,0,788,3657,0,...,776.0,32826.0,425.2,405.0,421.3,421.8,451073.0,255645.0,446930.0,447491.0
1,59,0004017380,1076,0,795,1046,0,9360,21,2,...,679.0,76638.0,526.0,167.2,310.7,499.6,1046934.0,94420.0,285645.0,907144.0
2,58,0004012746,128,0,155,1586,0,35,120,1804,...,761.0,45257.0,281.2,193.5,432.9,315.9,616894.0,95364.0,511875.0,575740.0
3,57,0003316510,1453,0,1192,3013,0,1618,274,3,...,504.0,35871.0,167.5,228.7,212.8,165.7,453092.0,131107.0,241543.0,324339.0
4,56,0003315900,500,0,0,2259,0,0,0,0,...,220.0,13119.0,140.5,368.7,151.1,155.1,180437.0,138644.0,155818.0,172558.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149,6138,9912128031,35,0,3141,3646,435,423,235,1157,...,127.4,100.0,162.9,123.6,257817.0,64578.0,145707.0,218865.0,NaN,NaN
6150,6137,9912108793,17,0,0,176,0,55,247,0,...,194.7,279.7,182.3,214.6,567568.0,148953.0,219701.0,447149.0,NaN,NaN
6151,6136,9912100253,42,0,152,1168,0,65,1147,150,...,131.2,91.6,116.8,122.5,365479.0,59859.0,153194.0,284911.0,NaN,NaN
6152,6135,9912089258,500,0,198,1956,0,88,1149,44,...,326.5,326.5,256.3,322.1,667121.0,127873.0,236328.0,473779.0,NaN,NaN


### Join Target with transactions

In [13]:
# Merge df with transactions to find non-matching SSNs
merged_df = pd.merge(df, transactions, on='SSN', how='inner', indicator=True)



In [14]:
transactions_filtered = transactions[~transactions['SSN'].isin(df['SSN'])]


In [15]:
transactions_filtered.tail(50)

,NR,SSN,avg_CashWithdrawal,avg_Crypto,avg_DebtCollection,avg_Deposit,avg_DirectDebit,avg_Gambling,avg_Gaming,avg_HousingCost,...,Unclassified_partOfSalary6Months,Unclassified_partOfSalary9Months,Unclassified_sum12Months,Unclassified_sum3Months,Unclassified_sum6Months,Unclassified_sum9Months,sum creditors,sum debtCollectors,count creditors,count debtCollectors
5692,4462,9401020483,259,0,0,1699,4849,143,40,10339,...,88.3,53.3,57.7,74.0,304376.0,51012.0,120851.0,212563.0,NaN,NaN
5697,4457,9312245955,2532,357,1717,4004,0,0,457,7833,...,442.8,475.9,465.7,478.8,1104975.0,312242.0,591220.0,911019.0,NaN,NaN
5715,4439,9311233028,21,0,3307,978,0,85,43,151,...,608.1,635.7,617.7,560.0,1536212.0,440736.0,877886.0,1184180.0,NaN,NaN
5720,4434,9311191697,1164,0,3420,0,0,269,385,2744,...,74.9,87.0,54.2,80.8,185608.0,47634.0,71836.0,157055.0,NaN,NaN
5721,4433,9311190368,167,0,153,3044,0,338,56,0,...,239.5,366.8,265.6,243.9,489214.0,178578.0,290715.0,401221.0,NaN,NaN
5729,4425,9311055314,23,0,0,1951,5470,354,273,8689,...,192.4,179.3,177.2,179.3,619826.0,135489.0,268752.0,406941.0,NaN,NaN
5731,4423,9310291613,0,0,14,0,0,0,0,3047,...,82.6,49.0,97.8,99.8,357222.0,53975.0,217847.0,323145.0,NaN,NaN
5740,4414,9310085353,0,0,26,1071,0,219,43,10,...,331.9,581.6,412.2,357.5,934843.0,406214.0,592618.0,762113.0,NaN,NaN
5741,4413,9310075719,5,0,0,25278,0,404,81,1558,...,185.1,440.8,273.3,216.2,547691.0,325181.0,400701.0,480847.0,NaN,NaN
5754,4400,9309153840,0,0,0,111,0,1336,189,0,...,147.7,190.3,169.2,165.1,229274.0,48466.0,86357.0,147043.0,NaN,NaN


In [16]:
transactions[transactions.SSN == '0003305547']

,NR,SSN,avg_CashWithdrawal,avg_Crypto,avg_DebtCollection,avg_Deposit,avg_DirectDebit,avg_Gambling,avg_Gaming,avg_HousingCost,...,Unclassified_partOfSalary6Months,Unclassified_partOfSalary9Months,Unclassified_sum12Months,Unclassified_sum3Months,Unclassified_sum6Months,Unclassified_sum9Months,sum creditors,sum debtCollectors,count creditors,count debtCollectors
5,55,0003305547,2317,0,1294,2494,0,37,125,0,...,890.0,45076.0,302.5,305.5,300.5,277.4,581852.0,182253.0,352434.0,483413.0


In [17]:
df[df.SSN == '0003305547']

,ReceivedDate,ApplicationID,AccountNumber,SSN,DisbursedDate,Amount,IsMainApplicant,ApplicantNo,HasCoapp,Ever90


In [18]:
merged_df = merged_df.drop_duplicates()

In [19]:
merged_df

,ReceivedDate,ApplicationID,AccountNumber,SSN,DisbursedDate,Amount,IsMainApplicant,ApplicantNo,HasCoapp,Ever90,...,Unclassified_partOfSalary9Months,Unclassified_sum12Months,Unclassified_sum3Months,Unclassified_sum6Months,Unclassified_sum9Months,sum creditors,sum debtCollectors,count creditors,count debtCollectors,_merge
0,2022-02-11,8185902,7720410,9110280782,2022-02-15,240000.0,1,1,0,0,...,17267.0,97.9,90.3,105.8,103.2,226735.0,56348.0,128736.0,183396.0,both
1,2022-02-21,8192526,7724719,9611102204,2022-02-25,105000.0,1,1,0,0,...,29.1,84.8,79.9,187603.0,16423.0,99416.0,144479.0,NaN,NaN,both
2,2022-03-01,8199216,7726243,9604274911,2022-03-02,146000.0,1,1,0,0,...,513.6,419.9,430.4,981711.0,110771.0,475430.0,778351.0,NaN,NaN,both
3,2022-03-01,8199738,7726482,9709124888,2022-03-03,80000.0,1,1,0,0,...,236.6,152.5,139.5,294364.0,139170.0,190490.0,253897.0,NaN,NaN,both
4,2022-03-02,8200529,7726854,8904272500,2022-03-11,125000.0,1,1,0,0,...,230.5,537430.0,186804.0,376038.0,465261.0,NaN,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4752,2023-11-26,8893962,7982648,6501038605,2023-11-29,115000.0,1,1,1,0,...,21777.0,133.5,155.1,128.9,130.7,285436.0,79645.0,141430.0,211217.0,both
4753,2023-11-27,8895474,7982663,8609217537,2023-11-29,361000.0,0,2,1,0,...,366.5,1702810.0,545095.0,873504.0,1059542.0,NaN,NaN,NaN,NaN,both
4754,2023-11-27,8895474,7982663,8612276900,2023-11-29,361000.0,1,1,1,0,...,766.4,250316.0,210976.0,217627.0,225101.0,NaN,NaN,NaN,NaN,both
4755,2023-11-29,8897063,7983174,9310119012,2023-11-30,190000.0,1,1,0,0,...,366.6,655.4,561.0,1737339.0,346243.0,1265976.0,1548267.0,NaN,NaN,both


In [20]:
merged_df[merged_df.AccountNumber == '7732407']

,ReceivedDate,ApplicationID,AccountNumber,SSN,DisbursedDate,Amount,IsMainApplicant,ApplicantNo,HasCoapp,Ever90,...,Unclassified_partOfSalary9Months,Unclassified_sum12Months,Unclassified_sum3Months,Unclassified_sum6Months,Unclassified_sum9Months,sum creditors,sum debtCollectors,count creditors,count debtCollectors,_merge
24,2022-03-17,8212497,7732407,9001144832,2022-03-18,300000.0,1,1,1,0,...,116.7,283294.0,66356.0,136740.0,209745.0,NaN,NaN,NaN,NaN,both


In [21]:
frame = merged_df.sort_values(by='ReceivedDate')

frame['ReceivedDate'] = pd.to_datetime(frame['ReceivedDate'])



target = frame[frame.Ever90 == 1]
rest = frame[frame.Ever90 == 0]
rest = rest[ (rest.ReceivedDate < pd.to_datetime('2023-05-01'))]

df = pd.concat([target,rest])

In [22]:

# List to save columns that contain string values
savestringcolumns = []

for c in df.columns:
    # Skip specified columns
    if c in ['ApplicationID', 'SSN']:
        continue

    # Check and handle string values
    df[c] = df[c].apply(lambda x: 0 if isinstance(x, str) else x)
    savestringcolumns.append(c) if df[c].dtype == 'object' else df[c].fillna(0, inplace=True)

# Print the modified DataFrame
print(df)

     ReceivedDate ApplicationID  AccountNumber         SSN DisbursedDate  \
13     2022-03-08       8205045              0  7711025523    2022-03-10   
55     2022-04-07       8228347              0  9804048719    2022-04-22   
51     2022-04-08       8229075              0  6501225061    2022-04-12   
50     2022-04-11       8230481              0  9606236371    2022-04-13   
218    2022-04-15       8233936              0  8602147608    2022-06-08   
...           ...           ...            ...         ...           ...   
3404   2023-04-28       8660675              0  8902289209    2023-05-31   
3405   2023-04-28       8660675              0  8903213695    2023-05-31   
3401   2023-04-28       8660600              0  8007075586    2023-05-08   
3402   2023-04-28       8660600              0  8203014934    2023-05-08   
3412   2023-04-30       8661965              0  9106092506    2023-05-09   

        Amount  IsMainApplicant  ApplicantNo  HasCoapp  Ever90  ...  \
13    250000.0  

In [23]:
df

,ReceivedDate,ApplicationID,AccountNumber,SSN,DisbursedDate,Amount,IsMainApplicant,ApplicantNo,HasCoapp,Ever90,...,Unclassified_partOfSalary9Months,Unclassified_sum12Months,Unclassified_sum3Months,Unclassified_sum6Months,Unclassified_sum9Months,sum creditors,sum debtCollectors,count creditors,count debtCollectors,_merge
13,2022-03-08,8205045,0,7711025523,2022-03-10,250000.0,1,1,0,1,...,50227.0,224.1,248.5,247.0,258.4,630444.0,81995.0,288551.0,519248.0,0
55,2022-04-07,8228347,0,9804048719,2022-04-22,400000.0,1,1,1,1,...,763.7,1333.3,1553.4,2678645.0,241491.0,1655157.0,2249127.0,0.0,0.0,0
51,2022-04-08,8229075,0,6501225061,2022-04-12,240000.0,1,1,0,1,...,85589.0,322.8,415.5,320.5,331.8,882268.0,279047.0,442904.0,685525.0,0
50,2022-04-11,8230481,0,9606236371,2022-04-13,295000.0,1,1,0,1,...,268.0,476.0,439.5,883993.0,100348.0,452428.0,627919.0,0.0,0.0,0
218,2022-04-15,8233936,0,8602147608,2022-06-08,270000.0,1,1,0,1,...,379.3,1362741.0,251386.0,547363.0,971946.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,2023-04-28,8660675,0,8902289209,2023-05-31,271000.0,1,1,1,0,...,195.3,381102.0,84024.0,274075.0,312523.0,0.0,0.0,0.0,0.0,0
3405,2023-04-28,8660675,0,8903213695,2023-05-31,271000.0,0,2,1,0,...,121.2,348937.0,91087.0,215240.0,265065.0,0.0,0.0,0.0,0.0,0
3401,2023-04-28,8660600,0,8007075586,2023-05-08,350000.0,1,1,1,0,...,40754.0,165.3,135.9,130.5,145.1,509094.0,147353.0,245526.0,367844.0,0
3402,2023-04-28,8660600,0,8203014934,2023-05-08,350000.0,0,2,1,0,...,38839.0,96.0,76.7,72.3,92.7,487593.0,113811.0,198074.0,353883.0,0


In [24]:
df

,ReceivedDate,ApplicationID,AccountNumber,SSN,DisbursedDate,Amount,IsMainApplicant,ApplicantNo,HasCoapp,Ever90,...,Unclassified_partOfSalary9Months,Unclassified_sum12Months,Unclassified_sum3Months,Unclassified_sum6Months,Unclassified_sum9Months,sum creditors,sum debtCollectors,count creditors,count debtCollectors,_merge
13,2022-03-08,8205045,0,7711025523,2022-03-10,250000.0,1,1,0,1,...,50227.0,224.1,248.5,247.0,258.4,630444.0,81995.0,288551.0,519248.0,0
55,2022-04-07,8228347,0,9804048719,2022-04-22,400000.0,1,1,1,1,...,763.7,1333.3,1553.4,2678645.0,241491.0,1655157.0,2249127.0,0.0,0.0,0
51,2022-04-08,8229075,0,6501225061,2022-04-12,240000.0,1,1,0,1,...,85589.0,322.8,415.5,320.5,331.8,882268.0,279047.0,442904.0,685525.0,0
50,2022-04-11,8230481,0,9606236371,2022-04-13,295000.0,1,1,0,1,...,268.0,476.0,439.5,883993.0,100348.0,452428.0,627919.0,0.0,0.0,0
218,2022-04-15,8233936,0,8602147608,2022-06-08,270000.0,1,1,0,1,...,379.3,1362741.0,251386.0,547363.0,971946.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,2023-04-28,8660675,0,8902289209,2023-05-31,271000.0,1,1,1,0,...,195.3,381102.0,84024.0,274075.0,312523.0,0.0,0.0,0.0,0.0,0
3405,2023-04-28,8660675,0,8903213695,2023-05-31,271000.0,0,2,1,0,...,121.2,348937.0,91087.0,215240.0,265065.0,0.0,0.0,0.0,0.0,0
3401,2023-04-28,8660600,0,8007075586,2023-05-08,350000.0,1,1,1,0,...,40754.0,165.3,135.9,130.5,145.1,509094.0,147353.0,245526.0,367844.0,0
3402,2023-04-28,8660600,0,8203014934,2023-05-08,350000.0,0,2,1,0,...,38839.0,96.0,76.7,72.3,92.7,487593.0,113811.0,198074.0,353883.0,0


In [26]:

exclude = ['ReceivedDate','ApplicationID','SSN','DisbursedDate','Amount','IsMainApplicant','ApplicantNo','NR']


# Function to check for columns with string values and return the columns along with unique string values
def find_columns_with_strings_and_values(df,exclude):
    string_columns = {}
    for col in df.columns:

        if col in exclude:
            continue

        string_values = set(df[col].apply(lambda x: x if isinstance(x, str) else None).dropna())
        if string_values:
            string_columns[col] = string_values
    return string_columns

# Find columns with string values and their unique string values
columns_with_strings_and_values = find_columns_with_strings_and_values(df,exclude)

# Print the results
for col, values in columns_with_strings_and_values.items():
    print(f"Column '{col}' contains string values: {values}")


In [30]:
df.to_excel('PreeProcessed_Psd2.xlsx', index=False)
